In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from tqdm import tqdm_notebook as tqdm

from tensorflow.keras.optimizers import Adam , RMSprop
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping , CSVLogger
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input , concatenate , Conv2D , MaxPooling2D , Activation , UpSampling2D , BatchNormalization, Conv2DTranspose, Add
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [ ]:
def dice_coeff(y_true , y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score


def dice_loss(y_true , y_pred):
    loss = 1- dice_coeff(y_true , y_pred)
    return loss


def bce_dice_loss(y_true , y_pred):
    loss = binary_crossentropy(y_true , y_pred) + dice_loss(y_true , y_pred)
    return loss

In [ ]:
def standard_uint(input_tensor, nb_filter):
    
    x = Conv2D(nb_filter, (3, 3), padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_filter, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x


def get_nest_unet_512_layer_5(input_shape=(512, 512, 1), num_classes=3, deep_supervision = False):
    
    with tf.device('/gpu:0'):
        
        inputs = Input(shape = input_shape)
        
        # 512
        conv1_1 = standard_uint(inputs, nb_filter = 16)
        pool1 = MaxPooling2D((2, 2), strides = (2, 2))(conv1_1)
        
        #256
        conv2_1 = standard_uint(pool1, nb_filter = 32)
        pool2 = MaxPooling2D((2, 2), strides = (2, 2))(conv2_1)
        
        up1_2 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_1)
        conv1_2 = concatenate([up1_2, conv1_1], axis = 3)
        conv1_2 = standard_uint(conv1_2, nb_filter = 16)
        
        #128
        conv3_1 = standard_uint(pool2, nb_filter = 64)
        pool3 = MaxPooling2D((2, 2), strides = (2, 2))(conv3_1)
        
        up2_2 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_1)
        conv2_2 = concatenate([up2_2, conv2_1], axis = 3)
        conv2_2 = standard_uint(conv2_2, nb_filter = 32)
        
        up1_3 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_2)
        conv1_3 = concatenate([up1_3, conv1_1, conv1_2], axis = 3)
        conv1_3 = standard_uint(conv1_3, nb_filter = 16)
        
        # 64
        conv4_1 = standard_uint(pool3, nb_filter = 128)
        pool4 = MaxPooling2D((2, 2), strides = (2, 2))(conv4_1)      
        
        up3_2 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_1)
        conv3_2 = concatenate([up3_2, conv3_1], axis = 3)
        conv3_2 = standard_uint(conv3_2, nb_filter = 64)    
        
        up2_3 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_2)
        conv2_3 = concatenate([up2_3, conv2_1, conv2_2], axis = 3)
        conv2_3 = standard_uint(conv2_3, nb_filter = 32)        
        
        up1_4 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_3)
        conv1_4 = concatenate([up1_4, conv1_2, conv1_3], axis = 3)
        conv1_4 = standard_uint(conv1_4, nb_filter = 16)       
        
        # 32
        conv5_1 = standard_uint(pool4, nb_filter = 256)    
        
        up4_2 = Conv2DTranspose(128, (2, 2), strides = (2, 2), padding = 'same')(conv5_1)
        conv4_2 = concatenate([up4_2, conv4_1], axis = 3)
        conv4_2 = standard_uint(conv4_2, nb_filter = 128)  
        
        up3_3 = Conv2DTranspose(64, (2, 2), strides = (2, 2), padding = 'same')(conv4_2)
        conv3_3 = concatenate([up3_3, conv3_1, conv3_2], axis = 3)
        conv3_3 = standard_uint(conv3_3, nb_filter = 64)
        
        up2_4 = Conv2DTranspose(32, (2, 2), strides = (2, 2), padding = 'same')(conv3_3)
        conv2_4 = concatenate([up2_4, conv2_1, conv2_2, conv2_3], axis = 3)
        conv2_4 = standard_uint(conv2_4, nb_filter = 32)
        
        up1_5 = Conv2DTranspose(16, (2, 2), strides = (2, 2), padding = 'same')(conv2_4)
        conv1_5 = concatenate([up1_5, conv1_1, conv1_2, conv1_3, conv1_4], axis = 3)
        conv1_5 = standard_uint(conv1_5, nb_filter = 16)
        
        
        
        nested_output_1 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_2)
        nested_output_2 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_3)
        nested_output_3 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_4)
        nested_output_4 = Conv2D(num_classes, (1, 1), activation = 'sigmoid')(conv1_5)
        
        if deep_supervision:
            model = Model(inputs = inputs, outputs = [nested_output_1, nested_output_2, nested_output_3, nested_output_4]) 
        else:
            model = Model(inputs = inputs, outputs = [nested_output_4])
            
        model.compile(optimizer=RMSprop(lr=0.0001), loss=bce_dice_loss, metrics=[dice_coeff])
        
        return model


In [ ]:
def normalize_x(image):
    return image / 127.5 - 1

def normalize_y(image):
    return image / 255

def denormalize_y(image):
    return image * 255

In [ ]:
def load_Y_gray_with_gaussian(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    print(image_files)
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + image_file ,
            cv2.IMREAD_GRAYSCALE
        )

        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )
                
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

def load_X_gray(folder_path):
    
    image_files = []       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    print (image_files)
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)

    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    return images, image_files


In [ ]:
def divide_gray_imgs(images):
    
    H = -(-images.shape[1]//412)
    W = -(-images.shape[2]//412)
    
    diveded_imgs = np.zeros(( images.shape[0]*H*W, 512, 512, 1), np.float32)
    print(H,W)
    
    for z in range(images.shape[0]):
        image = images[z]
        for h in range(H):
            for w in range(W):
                cropped_img = np.zeros((512, 512, 1), np.float32)
                cropped_img -= 1
                
                if images.shape[1] < 412:
                    h = -1
                if images.shape[2] < 412:
                    w = -1
                    
                if h == -1:
                    if w == -1:
                        cropped_img[50:images.shape[1]+50, 50:images.shape[2]+50, 0] = image[0:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:images.shape[1]+50, 50:512, 0] = image[0:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:images.shape[1]+50, 0:images.shape[2]-412*W-50, 0] = image[0:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        cropped_img[50:images.shape[1]+50, :, 0] = image[0:images.shape[1], w*412-50:(w+1)*412+50, 0]
                elif h == 0:
                    if w == -1:
                        cropped_img[50:512, 50:images.shape[2]+50, 0] = image[0:462, 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:512, 50:512, 0] = image[0:462, 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:512, 0:images.shape[2]-412*W-50, 0] = image[0:462, w*412-50:images.shape[2], 0]
                    else:
                        try:
                            cropped_img[50:512, :, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[50:512, 0:images.shape[2]-412*(W-1)-50, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                elif h == H-1:
                    if w == -1:
                        cropped_img[0:images.shape[1]-412*H-50, 50:images.shape[2]+50, 0] = image[h*412-50:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[0:images.shape[1]-412*H-50, 50:512, 0] = image[h*412-50:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        try:
                            cropped_img[0:images.shape[1]-412*H-50, :, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                else:
                    if w == -1:
                        cropped_img[:, 50:images.shape[2]+50, 0] = image[h*412-50:(h+1)*412+50, 0:images.shape[2], 0]
                    elif w == 0:
                        try:
                            cropped_img[:, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                    elif w == W-1:
                        try:
                            cropped_img[:, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                    else:
                        try:
                            cropped_img[:, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]         
                        except:
                            try:
                                 cropped_img[:, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                            except:
                                 cropped_img[0:images.shape[1]-412*(H-1)-50, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                h = max(0, h)
                w = max(0, w)
                diveded_imgs[z*H*W+ w*H+h] = cropped_img
                
    return diveded_imgs


def merge_imgs(imgs, original_image_shape):
    
    merged_imgs = np.zeros((original_image_shape[0], original_image_shape[1], original_image_shape[2], 1), np.float32)
    H = -(-original_image_shape[1]//412)
    W = -(-original_image_shape[2]//412)    
    
    for z in range(original_image_shape[0]):
        for h in range(H):
            for w in range(W):

                if original_image_shape[1] < 412:
                    h = -1
                if original_image_shape[2] < 412:
                    w = -1
 
                if h == -1:
                    if w == -1:
                        merged_imgs[z, 0:original_image_shape[1], 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+0][50:original_image_shape[1]+50, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, 0:original_image_shape[1], 0:412, 0] = imgs[z*H*W+ w*H+0][50:original_image_shape[1]+50, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, 0:original_image_shape[1], w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+0][50:original_image_shape[1]+50, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, 0:original_image_shape[1], w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+0][50:original_image_shape[1]+50, 50:462, 0]
                elif h == 0:
                    if w == -1:
                        merged_imgs[z, 0:412, 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+h][50:462, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, 0:412, 0:412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, 0:412, w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+h][50:462, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, 0:412, w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]
                elif h == H-1:
                    if w == -1:
                         merged_imgs[z, h*412:original_image_shape[1], 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+h][50:original_image_shape[1]-412*H-50, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, h*412:original_image_shape[1], 0:412, 0] = imgs[z*H*W+ w*H+h][50:original_image_shape[1]-412*H-50, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, h*412:original_image_shape[1], w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+h][50:original_image_shape[1]-412*H-50, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, h*412:original_image_shape[1], w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+h][50:original_image_shape[1]-412*H-50, 50:462, 0]
                else:
                    if w == -1:
                         merged_imgs[z, h*412:(h+1)*412, 0:original_image_shape[2], 0] = imgs[z*H*W+ 0*H+h][50:462, 50:original_image_shape[2]+50, 0]
                    elif w == 0:
                        merged_imgs[z, h*412:(h+1)*412, 0:412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]
                    elif w == W-1:
                        merged_imgs[z, h*412:(h+1)*412, w*412:original_image_shape[2], 0] = imgs[z*H*W+ w*H+h][50:462, 50:original_image_shape[2]-412*W-50, 0]
                    else:
                        merged_imgs[z, h*412:(h+1)*412, w*412:(w+1)*412, 0] = imgs[z*H*W+ w*H+h][50:462, 50:462, 0]  
                        
    return merged_imgs



def divide_imgs(images):
    
    H = -(-images.shape[1]//412)
    W = -(-images.shape[2]//412)
    
    diveded_imgs = np.zeros(( images.shape[0]*H*W, 512, 512, 1), np.float32)
    print(H,W)
    
    for z in range(images.shape[0]):
        image = images[z]
        for h in range(H):
            for w in range(W):
                cropped_img = np.zeros((512, 512, 1), np.float32)
                cropped_img -= 1
                
                if images.shape[1] < 412:
                    h = -1
                if images.shape[2] < 412:
                    w = -1
                    
                if h == -1:
                    if w == -1:
                        cropped_img[50:images.shape[1]+50, 50:images.shape[2]+50, 0] = image[0:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:images.shape[1]+50, 50:512, 0] = image[0:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:images.shape[1]+50, 0:images.shape[2]-412*W-50, 0] = image[0:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        cropped_img[50:images.shape[1]+50, :, 0] = image[0:images.shape[1], w*412-50:(w+1)*412+50, 0]
                elif h == 0:
                    if w == -1:
                        cropped_img[50:512, 50:images.shape[2]+50, 0] = image[0:462, 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[50:512, 50:512, 0] = image[0:462, 0:462, 0]
                    elif w == W-1:
                        cropped_img[50:512, 0:images.shape[2]-412*W-50, 0] = image[0:462, w*412-50:images.shape[2], 0]
                    else:
                        try:
                            cropped_img[50:512, :, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[50:512, 0:images.shape[2]-412*(W-1)-50, 0] = image[0:462, w*412-50:(w+1)*412+50, 0]
                elif h == H-1:
                    if w == -1:
                        cropped_img[0:images.shape[1]-412*H-50, 50:images.shape[2]+50, 0] = image[h*412-50:images.shape[1], 0:images.shape[2], 0]
                    elif w == 0:
                        cropped_img[0:images.shape[1]-412*H-50, 50:512, 0] = image[h*412-50:images.shape[1], 0:462, 0]
                    elif w == W-1:
                        cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:images.shape[1], w*412-50:images.shape[2], 0]
                    else:
                        try:
                            cropped_img[0:images.shape[1]-412*H-50, :, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:images.shape[1], w*412-50:(w+1)*412+50, 0]
                else:
                    if w == -1:
                        cropped_img[:, 50:images.shape[2]+50, 0] = image[h*412-50:(h+1)*412+50, 0:images.shape[2], 0]
                    elif w == 0:
                        try:
                            cropped_img[:, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 50:512, 0] = image[h*412-50:(h+1)*412+50, 0:462, 0]
                    elif w == W-1:
                        try:
                            cropped_img[:, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                        except:
                            cropped_img[0:images.shape[1]-412*H-50+412, 0:images.shape[2]-412*W-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:images.shape[2], 0]
                    else:
                        try:
                            cropped_img[:, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]         
                        except:
                            try:
                                 cropped_img[:, 0:images.shape[2]-412*(W-1)-50, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                            except:
                                 cropped_img[0:images.shape[1]-412*(H-1)-50, :, 0] = image[h*412-50:(h+1)*412+50, w*412-50:(w+1)*412+50, 0]
                h = max(0, h)
                w = max(0, w)
                diveded_imgs[z*H*W+ w*H+h] = cropped_img
                
    return diveded_imgs
    

In [ ]:
def make_train_datasets(ori_images, label_images, num_list):
    
    """
        Args : 
            ori_images (numpy.ndarray) : Z, Y, X, 1
            label_images (numpy.ndarray) : Z, Y, X, 1
            num_list (list) : num
            
        Returns : 
            train_ori_images (numpy.ndarray) : Z, Y, X, 1
            train_label_images (numpy.ndarray) : Z, Y, X, 1
        
    """
    
    train_ori_images = list()
    train_label_images = list()
    for num in num_list:
        ori_image = ori_images[num]
        label_image = label_images[num]

        x_range = [int(ori_image.shape[0] * 0.1), int(ori_image.shape[0] * 0.9)]
        y_range = [int(ori_image.shape[1] * 0.1), int(ori_image.shape[1] * 0.9)]
        
        train_ori_images.append(ori_image[x_range[0]:x_range[1], y_range[0]:y_range[1], :])
        train_label_images.append(label_image[x_range[0]:x_range[1], y_range[0]:y_range[1], :])
        
    cropped_ori_images = divide_gray_imgs(np.array(train_ori_images))
    cropped_label_images = divide_gray_imgs(np.array(train_label_images))
    
    cropped_label_images = np.where(
        cropped_label_images == -1,
        0,
        cropped_label_images
    )
        
    return cropped_ori_images, cropped_label_images

In [ ]:
def train_unet(X_train,Y_train, csv_path, model_path ,input_shape=(512, 512, 1), num_classes=1):
    Y_train = Y_train
    X_train = X_train
    
    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    data_gen_args = dict(
        rotation_range=90.,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True
    )
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    seed = 1
    image_datagen.fit(X_train, augment=True, seed=seed)
    mask_datagen.fit(Y_train, augment=True, seed=seed)

    image_generator = image_datagen.flow(X_train, seed=seed, batch_size=8)
    mask_generator = mask_datagen.flow(Y_train, seed=seed, batch_size=8)

    train_generator = (pair for pair in zip(image_generator, mask_generator))
    model = get_nest_unet_512_layer_5(input_shape=input_shape, num_classes=num_classes)

    BATCH_SIZE = 4
    NUM_EPOCH = 400
    
    callbacks = []
    from tensorflow.keras.callbacks import CSVLogger
    callbacks.append(CSVLogger(csv_path))
    history = model.fit_generator(train_generator,steps_per_epoch=32, epochs=NUM_EPOCH, verbose=1, callbacks=callbacks)
    model.save_weights(model_path)

In [ ]:
def predict(X_test, model_path, filenames, out_dir, input_shape=(512, 512, 1), num_classes=1):
    
    model = get_nest_unet_512_layer_5(input_shape=input_shape, num_classes=num_classes)
    
    model.load_weights(model_path)
    BATCH_SIZE = 1
    Y_pred = model.predict(X_test, BATCH_SIZE)
    
    print(Y_pred.shape)
    os.makedirs(out_dir, exist_ok = True)
    
    if Y_pred.shape[3]!=1:
        num = Y_pred.shape[3]
        for n in range(num):
            os.makedirs(os.path.join(out_dir,str(n+1)), exist_ok=True)
        for i, y in enumerate(Y_pred):
            for n in range(num):
                os.makedirs(os.path.join(out_dir,str(n+1)), exist_ok=True)
                cv2.imwrite(os.path.join(out_dir, str(n+1) , str(i).zfill(4) + '.png'), denormalize_y(y[:,:,n]))
        
    else:
        for i, y in enumerate(Y_pred):
            cv2.imwrite(os.path.join(out_dir , str(i).zfill(4) + '.png'), denormalize_y(y))

# train_001

In [ ]:
ori_imgs , _ = load_X_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/dataset/cropped_002/resize_10x10x10/')
label_imgs_001 , _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/manually_mito_001//', normalize=False)


In [ ]:
num_lst_001 = [50, 150, 250]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)

In [ ]:
csv_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/mito_nested_unet_001.csv'
X_train = cropped_ori_001_imgs
Y_train = cropped_label_001_imgs
model_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

# pred_001

In [ ]:
model_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/mito_nested_unet_001_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_xy_001/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_yz_001/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_zx_001/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_zx_imgs, X_test


In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape

In [ ]:
mito_imgs_xy, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_xy_001')
merged_mito_imgs_xy = merge_imgs(mito_imgs_xy, ori_image_shape)
del mito_imgs_xy

mito_imgs_yz, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_yz_001')
merged_mito_imgs_yz = merge_imgs(mito_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del mito_imgs_yz

mito_imgs_zx, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_zx_001')
merged_mito_imgs_zx = merge_imgs(mito_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del mito_imgs_zx


In [ ]:
mito_imgs_ave = merged_mito_imgs_xy * 255 // 3 + merged_mito_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_mito_imgs_zx.transpose(2, 0, 1, 3) * 255 //3

out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/merged_001'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

In [ ]:
out_dir = "H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/manually_mito_002"
os.makedirs(out_dir, exist_ok = True)


# train_002

In [ ]:
ori_imgs , _ = load_X_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/dataset/cropped_002/resize_10x10x10/')
label_imgs_001 , _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/manually_mito_001//', normalize=False)
label_imgs_002, _ = load_Y_gray_with_gaussian("H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/manually_mito_002/",normalize=False)

In [ ]:
ori_80_imgs , _ = load_X_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002/original//')


In [ ]:
num_lst_001 = [50, 150, 250]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)

In [ ]:
num_lst_002 = [54, 170, 226]

cropped_ori_002_imgs, cropped_label_002_imgs  = make_80_train_datasets(ori_80_imgs,  label_imgs_002, num_lst_002)

In [ ]:
cropped_ori_imgs = np.concatenate([cropped_ori_001_imgs, cropped_ori_002_imgs],  axis=0)
cropped_label_imgs = np.concatenate([cropped_label_001_imgs, cropped_label_002_imgs], axis=0)

In [ ]:
csv_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/mito_nested_unet_002.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

# pred_002

In [ ]:
model_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/mito_nested_unet_002_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_xy_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_yz_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_zx_002/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_zx_imgs, X_test


In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape

In [ ]:
mito_imgs_xy, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_xy_002')
merged_mito_imgs_xy = merge_imgs(mito_imgs_xy, ori_image_shape)
del mito_imgs_xy

mito_imgs_yz, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_yz_002')
merged_mito_imgs_yz = merge_imgs(mito_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del mito_imgs_yz

mito_imgs_zx, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_zx_002')
merged_mito_imgs_zx = merge_imgs(mito_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del mito_imgs_zx


In [ ]:
mito_imgs_ave = merged_mito_imgs_xy * 255 // 3 + merged_mito_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_mito_imgs_zx.transpose(2, 0, 1, 3) * 255 //3

out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/merged_002'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

In [ ]:
out_dir = "H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/manually_mito_003"
os.makedirs(out_dir, exist_ok = True)


# train_003

In [ ]:
ori_imgs , _ = load_X_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/dataset/cropped_002/resize_10x10x10/')
ori_80_imgs , _ = load_X_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002/original//')

label_imgs_001 , _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/manually_mito_001//', normalize=False)
label_imgs_002, _ = load_Y_gray_with_gaussian("H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/manually_mito_002/",normalize=False)
label_imgs_003,  _ = load_Y_gray_with_gaussian("H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/manually_mito_003/",normalize=False)


In [ ]:
num_lst_001 = [50, 150, 250]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)

num_lst_002 = [54, 170, 226]
num_lst_003 = [57, 92, 166, 202]

cropped_ori_002_imgs, cropped_label_002_imgs  = make_80_train_datasets(ori_80_imgs,  label_imgs_002, num_lst_002)
cropped_ori_003_imgs, cropped_label_003_imgs  = make_80_train_datasets(ori_80_imgs,  label_imgs_003, num_lst_003)


In [ ]:
cropped_ori_imgs = np.concatenate([cropped_ori_001_imgs, cropped_ori_002_imgs, cropped_ori_003_imgs],  axis=0)
cropped_label_imgs = np.concatenate([cropped_label_001_imgs, cropped_label_002_imgs, cropped_label_003_imgs], axis=0)

In [ ]:
csv_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/mito_nested_unet_003.csv'
X_train = cropped_ori_imgs
Y_train = cropped_label_imgs
model_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/mito_nested_unet_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

# pred_003

In [ ]:
model_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/mito_nested_unet_003_weights.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_xy_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_yz_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_zx_003/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_zx_imgs, X_test


In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape

In [ ]:
mito_imgs_xy, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_xy_003')
merged_mito_imgs_xy = merge_imgs(mito_imgs_xy, ori_image_shape)
del mito_imgs_xy

mito_imgs_yz, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_yz_003')
merged_mito_imgs_yz = merge_imgs(mito_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del mito_imgs_yz

mito_imgs_zx, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/pred_zx_003')
merged_mito_imgs_zx = merge_imgs(mito_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del mito_imgs_zx


In [ ]:
mito_imgs_ave = merged_mito_imgs_xy * 255 // 3 + merged_mito_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_mito_imgs_zx.transpose(2, 0, 1, 3) * 255 //3

out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/merged_003'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

# without HITL train

In [ ]:
ori_imgs , _ = load_X_gray('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/dataset/cropped_002/resize_10x10x10/')
label_imgs_001 , _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/manually_mito_001//', normalize=False)


In [ ]:
num_lst_001 = [25, 50, 75, 100, 125, 150, 175, 200, 225, 250]

cropped_ori_001_imgs, cropped_label_001_imgs = make_train_datasets(ori_imgs, label_imgs_001, num_lst_001)

In [ ]:
csv_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/without_HITL_mito_nested_unet_001.csv'
X_train = cropped_ori_001_imgs
Y_train = cropped_label_001_imgs
model_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/without_HITL_mito_nested_unet_001.hdf5'
input_shape=(512, 512, 1)
num_classes=1

In [ ]:
train_unet(X_train,Y_train, csv_path, model_path, input_shape,num_classes)

# without HITL pred

In [ ]:
model_path = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/models/cropped_002_002/active_learning/without_HITL_mito_nested_unet_001.hdf5'
input_shape=(512, 512, 1)
num_classes=1
filenames = _

In [ ]:
seped_xy_imgs = divide_imgs(ori_imgs)

X_test = seped_xy_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_without_HITL/pred_xy_001/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_xy_imgs, X_test


seped_yz_imgs = divide_imgs(ori_imgs.transpose(2, 0, 1, 3))

X_test = seped_yz_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_without_HITL/pred_yz_001/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_yz_imgs, X_test


seped_zx_imgs = divide_imgs(ori_imgs.transpose(1, 2, 0, 3))

X_test = seped_zx_imgs
out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_without_HITL/pred_zx_001/'
predict(X_test, model_path, filenames, out_dir, input_shape, num_classes)

del  seped_zx_imgs, X_test


In [ ]:
ori_image_shape = ori_imgs.shape
del ori_imgs
ori_image_shape

In [ ]:
mito_imgs_xy, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_without_HITL/pred_xy_001')
merged_mito_imgs_xy = merge_imgs(mito_imgs_xy, ori_image_shape)
del mito_imgs_xy

mito_imgs_yz, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_without_HITL/pred_yz_001')
merged_mito_imgs_yz = merge_imgs(mito_imgs_yz, (ori_image_shape[2], ori_image_shape[0], ori_image_shape[1], ori_image_shape[3]))
del mito_imgs_yz

mito_imgs_zx, _ = load_Y_gray_with_gaussian('H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_without_HITL/pred_zx_001')
merged_mito_imgs_zx = merge_imgs(mito_imgs_zx, (ori_image_shape[1], ori_image_shape[2], ori_image_shape[0], ori_image_shape[3]))
del mito_imgs_zx


In [ ]:
mito_imgs_ave = merged_mito_imgs_xy * 255 // 3 + merged_mito_imgs_yz.transpose(1, 2, 0, 3) * 255 // 3 + merged_mito_imgs_zx.transpose(2, 0, 1, 3) * 255 //3

out_dir = 'H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_without_HITL/merged_001'
os.makedirs(out_dir, exist_ok = True)
os.makedirs(f"{out_dir}_raw", exist_ok = True)

z_range = [int(ori_image_shape[0] * 0.1), int(ori_image_shape[0] * 0.9)]
y_range = [int(ori_image_shape[1] * 0.1), int(ori_image_shape[1] * 0.9)]
x_range = [int(ori_image_shape[2] * 0.1), int(ori_image_shape[2] * 0.9)]


cnt = 0
for i in range(z_range[0], z_range[1]):
    
    # threshed
    img = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        1,
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}/{str(cnt).zfill(4)}.png', img)
    
    # averaged
    img_ = np.zeros(
        (y_range[1] - y_range[0], x_range[1] - x_range[0], 1), np.float32
    )
    img_ = np.where(
        mito_imgs_ave[:,:,:,0][i] >= 127,
        mito_imgs_ave[:,:,:,0][i],
        0
    )[y_range[0]:y_range[1], x_range[0]:x_range[1]]
    cv2.imwrite(f'{out_dir}_raw/{str(cnt).zfill(4)}.png', img_)
    
    cnt += 1

# culc F1 score and create representative imgs

In [ ]:
def make_mask_img(ori_img, mask_img):
    mask_img_rgb = np.zeros((mask_img.shape[0], mask_img.shape[1], 3), np.float32)
    mask_img_rgb[:,:,0] = mask_img[:,:,0]
    mask_img_rgb[:,:,2] = mask_img[:,:,0]
    masked_img = cv2.addWeighted(mask_img_rgb,0.5,cv2.cvtColor(ori_img+0.75, cv2.COLOR_GRAY2BGR),0.6,0)
    return masked_img


def make_two_mask_img(ori_img, mask_img, raw_img):
    mask_img_rgb = np.zeros((mask_img.shape[0], mask_img.shape[1], 3), np.float32)
    mask_img_rgb[:,:,2] = mask_img[:,:,0]
    mask_img_rgb[:,:,0] = mask_img[:,:,0]
    mask_img_rgb[:,:,1] += np.where(
        (raw_img[:,:,0]* 255 < 170) & (raw_img[:,:,0]* 255 > 127),
        1,
        0
    )
    mask_img_rgb[:,:,2] -= mask_img_rgb[:,:,1]
    mask_img_rgb[:,:,0] -= mask_img_rgb[:,:,1]
    
    masked_img = cv2.addWeighted(mask_img_rgb,0.5,cv2.cvtColor(ori_img+0.75, cv2.COLOR_GRAY2BGR),0.6,0)
    return masked_img

In [ ]:
def culc_f1(pred, gt):
    
    PRE = np.count_nonzero((pred*255 > 0.5) & (gt* 255 > 0.5)) / np.count_nonzero(pred*255 > 0.5)
    RE = np.count_nonzero((pred*255 > 0.5) & (gt * 255 > 0.5)) / np.count_nonzero(gt*255 > 0.5)
    
    try:
        F1 = 2 * (PRE * RE) / (PRE + RE)
        print(f"Precision: {round(PRE,2)}, Recall: {round(RE,2)}, F1: {round(F1,2)} ")
        return round(F1,2)
    except ZeroDivisionError:
        return 0
    except:
        return "error"

In [ ]:
ori_80_imgs , _ = load_X_gray('I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002/original//')

pred_imgs_001, _ = load_Y_gray_with_gaussian("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/merged_001/", normalize=False)
pred_imgs_002, _ = load_Y_gray_with_gaussian("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/merged_002/", normalize=False)
pred_imgs_003, _ = load_Y_gray_with_gaussian("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/merged_003/", normalize=False)

ori_imgs, _ = load_X_gray("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/dataset/cropped_002/resize_10x10x10/")
wo_pred_imgs, _ = load_Y_gray_with_gaussian("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_without_HITL/merged_001/", normalize=False)
mito_imgs, _ = load_Y_gray_with_gaussian("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/annotations/cropped_002/devided_mito_/", normalize=False)


In [ ]:
pred_imgs_001_raw, _ = load_Y_gray_with_gaussian("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/merged_001_raw//", normalize=False)
pred_imgs_002_raw, _ = load_Y_gray_with_gaussian("I:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_002_002/merged_002_raw//", normalize=False)


In [ ]:
N = 54
y = 850
x =  300

N_ = N +30
y_ = y + 143
x_ = x + 120


ori_054 = ori_80_imgs[N][y:y+150, x:x+150]
pred_001_054 = pred_imgs_001[N][y:y+150, x:x+150]
pred_002_054 = pred_imgs_002[N][y:y+150, x:x+150]
pred_003_054 = pred_imgs_003[N][y:y+150, x:x+150]
pred_wo_054 = wo_pred_imgs[N][y:y+150, x:x+150]

mito_054 = mito_imgs[N_][y_:y_+150, x_:x_+150]
first_054 = make_two_mask_img(
    ori_054,
    pred_001_054,
    pred_imgs_001_raw[N][y:y+150, x:x+150] / 255
)
second_054 = make_mask_img(
    ori_054,
    pred_002_054
)
third_054 = make_mask_img(
    ori_054,
    pred_003_054
)
wo_054 = make_mask_img(
    ori_054,
    pred_wo_054
)
gt_054 = make_mask_img(
    ori_054,
    mito_054
)
original_054 = make_mask_img(
    ori_054,
    np.zeros((150, 150, 1), np.float32)
)
"""
out_dir = "H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/for_paper/cropped_002_002/first_054_"

os.makedirs(out_dir, exist_ok = True)

cv2.imwrite(
    f"{out_dir}/000.png",
    denormalize_y(original_054)
)

cv2.imwrite(
    f"{out_dir}/001.png",
    denormalize_y(first_054)
)

cv2.imwrite(
    f"{out_dir}/002.png",
    denormalize_y(second_054)
)

cv2.imwrite(
    f"{out_dir}/003.png",
    denormalize_y(third_054)
)

cv2.imwrite(
    f"{out_dir}/004.png",
    denormalize_y(wo_054)
)

cv2.imwrite(
    f"{out_dir}/005.png",
    denormalize_y(gt_054)
)
"""
FIRST = culc_f1(pred_001_054, mito_054)
print(FIRST)

SECOND = culc_f1(pred_002_054, mito_054)
print(SECOND)

THIRD = culc_f1(pred_003_054, mito_054)
print(THIRD)

DL = culc_f1(pred_wo_054, mito_054)
print(DL)

In [ ]:
N = 92
y = 800
x =  350

N_ = N +30
y_ = y + 143
x_ = x + 120


ori_054 = ori_80_imgs[N][y:y+150, x:x+150]
pred_001_054 = pred_imgs_001[N][y:y+150, x:x+150]
pred_002_054 = pred_imgs_002[N][y:y+150, x:x+150]
pred_003_054 = pred_imgs_003[N][y:y+150, x:x+150]
pred_wo_054 = wo_pred_imgs[N][y:y+150, x:x+150]
mito_054 = mito_imgs[N_][y_:y_+150, x_:x_+150]

second_054 = make_two_mask_img(
    ori_054,
    pred_002_054,
    pred_imgs_002_raw[N][y:y+150, x:x+150] / 255
)

first_054 = make_mask_img(
    ori_054,
    pred_001_054
)

third_054 = make_mask_img(
    ori_054,
    pred_003_054
)

wo_054 = make_mask_img(
    ori_054,
    pred_wo_054
)

gt_054 = make_mask_img(
    ori_054,
    mito_054
)

original_054 = make_mask_img(
    ori_054,
    np.zeros((150, 150, 1), np.float32)
)
"""
out_dir = "H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/for_paper/cropped_002_002/second_092_"

os.makedirs(out_dir, exist_ok = True)

cv2.imwrite(
    f"{out_dir}/000.png",
    denormalize_y(original_054)
)

cv2.imwrite(
    f"{out_dir}/001.png",
    denormalize_y(first_054)
)

cv2.imwrite(
    f"{out_dir}/002.png",
    denormalize_y(second_054)
)

cv2.imwrite(
    f"{out_dir}/003.png",
    denormalize_y(third_054)
)

cv2.imwrite(
    f"{out_dir}/004.png",
    denormalize_y(wo_054)
)

cv2.imwrite(
    f"{out_dir}/005.png",
    denormalize_y(gt_054)
)
"""
FIRST = culc_f1(pred_001_054, mito_054)
print(FIRST)

SECOND = culc_f1(pred_002_054, mito_054)
print(SECOND)

THIRD = culc_f1(pred_003_054, mito_054)
print(THIRD)

DL = culc_f1(pred_wo_054, mito_054)
print(DL)

In [ ]:
N = 42
y = 850
x =  300

N_ = N +30
y_ = y + 143
x_ = x + 120


ori_054 = ori_80_imgs[N][y:y+150, x:x+150]
pred_001_054 = pred_imgs_001[N][y:y+150, x:x+150]
pred_002_054 = pred_imgs_002[N][y:y+150, x:x+150]
pred_003_054 = pred_imgs_003[N][y:y+150, x:x+150]
pred_wo_054 = wo_pred_imgs[N][y:y+150, x:x+150]
mito_054 = mito_imgs[N_][y_:y_+150, x_:x_+150]

first_054 = make_mask_img(
    ori_054,
    pred_001_054
)

second_054 = make_mask_img(
    ori_054,
    pred_002_054
)

third_054 = make_mask_img(
    ori_054,
    pred_003_054
)

wo_054 = make_mask_img(
    ori_054,
    pred_wo_054
)

gt_054 = make_mask_img(
    ori_054,
    mito_054
)

original_054 = make_mask_img(
    ori_054,
    np.zeros((150, 150, 1), np.float32)
)

"""
out_dir = "H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/for_paper/cropped_002_002/40_"

os.makedirs(out_dir, exist_ok = True)

cv2.imwrite(
    f"{out_dir}/000.png",
    denormalize_y(original_054)
)

cv2.imwrite(
    f"{out_dir}/001.png",
    denormalize_y(first_054)
)

cv2.imwrite(
    f"{out_dir}/002.png",
    denormalize_y(second_054)
)

cv2.imwrite(
    f"{out_dir}/003.png",
    denormalize_y(third_054)
)

cv2.imwrite(
    f"{out_dir}/004.png",
    denormalize_y(wo_054)
)

cv2.imwrite(
    f"{out_dir}/005.png",
    denormalize_y(gt_054)
)"""

FIRST = culc_f1(pred_001_054, mito_054)
print(FIRST)

SECOND = culc_f1(pred_002_054, mito_054)
print(SECOND)

THIRD = culc_f1(pred_003_054, mito_054)
print(THIRD)

DL = culc_f1(pred_wo_054, mito_054)
print(DL)

In [ ]:
N = 70
y = 825
x =  320

N_ = N +30
y_ = y + 143
x_ = x + 120


ori_054 = ori_80_imgs[N][y:y+150, x:x+150]
pred_001_054 = pred_imgs_001[N][y:y+150, x:x+150]
pred_002_054 = pred_imgs_002[N][y:y+150, x:x+150]
pred_003_054 = pred_imgs_003[N][y:y+150, x:x+150]
pred_wo_054 = wo_pred_imgs[N][y:y+150, x:x+150]
mito_054 = mito_imgs[N_][y_:y_+150, x_:x_+150]

first_054 = make_mask_img(
    ori_054,
    pred_001_054
)

second_054 = make_mask_img(
    ori_054,
    pred_002_054
)

third_054 = make_mask_img(
    ori_054,
    pred_003_054
)

wo_054 = make_mask_img(
    ori_054,
    pred_wo_054
)

gt_054 = make_mask_img(
    ori_054,
    mito_054
)

original_054 = make_mask_img(
    ori_054,
    np.zeros((150, 150, 1), np.float32)
)
"""

out_dir = "H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/for_paper/cropped_002_002/70_"

os.makedirs(out_dir, exist_ok = True)

cv2.imwrite(
    f"{out_dir}/000.png",
    denormalize_y(original_054)
)

cv2.imwrite(
    f"{out_dir}/001.png",
    denormalize_y(first_054)
)

cv2.imwrite(
    f"{out_dir}/002.png",
    denormalize_y(second_054)
)

cv2.imwrite(
    f"{out_dir}/003.png",
    denormalize_y(third_054)
)

cv2.imwrite(
    f"{out_dir}/004.png",
    denormalize_y(wo_054)
)

cv2.imwrite(
    f"{out_dir}/005.png",
    denormalize_y(gt_054)
)"""

FIRST = culc_f1(pred_001_054, mito_054)
print(FIRST)

SECOND = culc_f1(pred_002_054, mito_054)
print(SECOND)

THIRD = culc_f1(pred_003_054, mito_054)
print(THIRD)

DL = culc_f1(pred_wo_054, mito_054)
print(DL)